In [58]:
pip install yfinance --upgrade --no-cache-dir

In [59]:
from tensorflow import keras
import yfinance as yf
import numpy as np

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
cd /content/drive/MyDrive/Trading Bot

/content/drive/MyDrive/Trading Bot


In [62]:
from datetime import date, timedelta
today_date = date.today()
N = 10

In [63]:
file1 = open('company_list.txt',"r+") 
company_list = file1.read()
company_list = company_list.split('\n')
for i in range(len(company_list)):
    company_list[i] += '.NS'

In [64]:
model = keras.models.load_model('model')

In [65]:
def extractLast10Days(stock):
  hist = stock.history(start=today_date-timedelta(days=2*N), end=today_date)
  hist = hist["Close"]

  i = len(hist)-N
  price_list = []
  
  while len(price_list) < N:
    price_list.append(hist[i])
    i+=1

  return np.array([price_list])

def stockPredict(stock,n=5):
  price_list = extractLast10Days(stock)
  predictedPriceIndividualDays = []
  predictedPriceIndividualDays.append(price_list[0][-1])
  for i in range(n):
    output = model.predict(price_list)
    for j in range(len(price_list[0])-1):
        price_list[0][j] = price_list[0][j+1]
    price_list[0][len(price_list[0])-1] = output
    predictedPriceIndividualDays.append(output)

  return predictedPriceIndividualDays

def percentage_return():
  result = []
  for i in range(len(company_list)):
    stock = yf.Ticker(company_list[i])
    if(stock.info['logo_url']!=''):
      predict = stockPredict(stock)
      result.append((predict[-1]-predict[0])/predict[0])  
    else:
      result.append(float('NaN'))

  return result

In [66]:
ans = percentage_return()

In [71]:
ans = np.array(ans)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [78]:
company_list[ans.argmax()]

'COALINDIA.NS'